In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from segmentflow import segment, view
from skimage import measure
%load_ext autoreload
%autoreload 2

In [ ]:
def grading_curve(imgs_labeled, diameter_bins, pixel_res):
    # Collect sieve data
    # sieve_df = pd.read_csv(
    #     Path('../data/F50-sieve.csv'), index_col=0).sort_values('um')
    # diameter_ums = sieve_df.um.to_numpy()
    # diameter_ums_bins = np.insert(diameter_ums, 0, 0)
    # f50_pct = sieve_df['pct-retained'].to_numpy()
    # Format segmented data
    labels_df = pd.DataFrame(measure.regionprops_table(
        imgs_labeled, properties=['label', 'area', 'bbox']))
    labels_df = labels_df.rename(columns={'area' : 'volume'})
    # Calculate nslices by subtracting z min from max
    labels_df['nslices'] = (
        labels_df['bbox-3'].to_numpy() - labels_df['bbox-0'].to_numpy())
    # Calculate nrows by subtracting y min from max
    labels_df['nrows'] = (
        labels_df['bbox-4'].to_numpy() - labels_df['bbox-1'].to_numpy())
    # Calculate ncols by subtracting x min from max
    labels_df['ncols'] = (
        labels_df['bbox-5'].to_numpy() - labels_df['bbox-2'].to_numpy())
    # Define dimensions a, b, c with a as largest and c as smallest
    labels_df['a'] = labels_df.apply(
        lambda row: row['nslices' : 'ncols'].nlargest(3).iloc[0], axis=1)
    labels_df['b'] = labels_df.apply(
        lambda row: row['nslices' : 'ncols'].nlargest(3).iloc[1], axis=1)
    labels_df['c'] = labels_df.apply(
        lambda row: row['nslices' : 'ncols'].nlargest(3).iloc[2], axis=1)
    # Apply pixel resolution to second smallest dimension
    b_ums = pixel_res * labels_df['b'].to_numpy()
    hist, bins = np.histogram(b_ums, bins=diameter_bins)
    # seg_pct = 100 * seg_hist / labels_df.shape[0]
    # sieve_df[f'seg-aspect-pct'] = seg_pct
    return hist

def grading_curve(hist, diameter_bins):
    # seg_pct = 100 * seg_hist / labels_df.shape[0]
    hist_pct = 100 * hist / len(diameter_bins)
    # Plot histogram
    fig, ax = plt.subplots(
        figsize=(8, 5), facecolor='white', constrained_layout=True, dpi=300)
    # ax.scatter(diameter_ums, np.cumsum(f50_pct), s=10, zorder=3)
    # ax.plot(
    #     diameter_ums, np.cumsum(f50_pct), linewidth=2, zorder=3, label='F50 Standard')
    # Plot segmented particle size distributions
    # seg_i_pct = sieve_df[f'seg-aspect-pct'].to_numpy()
    # seg_i_pct_cum = np.cumsum(seg_i_pct)
    hist_pct_cum = np.cumsum(hist_pct)
    ax.scatter(diameter_bins, np.cumsum(hist_pct), s=10, zorder=2)
    ax.plot(diameter_bins, np.cumsum(hist_pct), linewidth=1, zorder=2)
    ax.set_title('Segmented Particle Size Distribution by Aspect Ratio')
    ax.set_ylabel(r'% retained on sieve')
    ax.set_ylim([0, 111])
    ax.set_xlabel('Particle diameter ($\mu m$)')
    ax.set_xscale('log')
    ax.grid(True, axis='y', zorder=0)
    ax.set_xlim([53, 850])
    # for v in np.concatenate(
    #     (np.arange(60, 100, 10, dtype=int), np.arange(100, 900, 100, dtype=int))):
    #     ax.axvline(v, linewidth=1, c='k', alpha=0.25, zorder=0)
    ax.set_xticks(diameter_bins)
    ax.set_xticklabels(diameter_bins)
    ax.legend(loc='lower right', ncol=1)
    return fig, ax